In [34]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import optuna
from hyperopt import fmin, hp, tpe, Trials

In [2]:
df = pd.read_csv('data/_train_sem09 (1).csv')
df.shape

(3751, 1777)

In [3]:
X = df.drop('Activity', axis=1)
y = df['Activity']

In [4]:
y.value_counts() # классы сбалансированы

Activity
1    2034
0    1717
Name: count, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Оптимизация Логистической регрессии

In [39]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)

log_reg.fit(X_train, y_train)

y_pred_log = log_reg.predict(X_test)

print('Значение f1_score на тестовых данных для дефорлной модели {}'
      .format(round(f1_score(y_test, y_pred_log), 2))
      )

Значение f1_score на тестовых данных для дефорлной модели 0.79


Оптимизация с помощью GridSearchCV

In [25]:
param_lin_reg_dict = {'penalty': ['l2', 'none'],
                    'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
                    'C': list(np.linspace(0.01, 1, 10, dtype=float))
}

grid_lin_reg = GridSearchCV(
        estimator=LogisticRegression(
        max_iter=1000,
        random_state=42),
    param_grid=param_lin_reg_dict,
    cv=5,
    n_jobs=-1
)

%time grid_lin_reg.fit(X_train, y_train)

CPU times: total: 18.2 s
Wall time: 17min 23s


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.12, 0.23, 0.34, 0.45, 0.56, 0.67, 0.78,
                               0.89, 1.0],
                         'penalty': ['l2', 'none'],
                         'solver': ['lbfgs', 'newton-cg', 'newton-cholesky',
                                    'sag', 'saga']})

In [26]:
grid_lin_reg.best_params_ # выводим лучшие параметры с использованием GridSearch

{'C': 0.12, 'penalty': 'l2', 'solver': 'sag'}

In [35]:
y_linreg_grid_search_predict = grid_lin_reg.predict(X_test)

print('Метрика f1 Логистической регрессии с применение GridSearch {}'
      .format(round(f1_score(y_test, y_linreg_grid_search_predict), 2)))

Метрика f1 Логистической регрессии с применение GridSearch 0.79


Оптимизация с помощью RandomizeSearchCV

In [32]:
random_search_lin_reg = RandomizedSearchCV(
    estimator=LogisticRegression(
        max_iter=1000, 
        random_state=42),
    param_distributions=param_lin_reg_dict,
    cv=5,
    n_iter=10,
    n_jobs=-1
)

%time random_search_lin_reg.fit(X_train, y_train)

CPU times: total: 3.61 s
Wall time: 1min 13s


RandomizedSearchCV(cv=5,
                   estimator=LogisticRegression(max_iter=1000, random_state=42),
                   n_jobs=-1,
                   param_distributions={'C': [0.01, 0.12, 0.23, 0.34, 0.45,
                                              0.56, 0.67, 0.78, 0.89, 1.0],
                                        'penalty': ['l2', 'none'],
                                        'solver': ['lbfgs', 'newton-cg',
                                                   'newton-cholesky', 'sag',
                                                   'saga']})

In [33]:
random_search_lin_reg.best_params_  # лучшие параметры с использование random search

{'solver': 'newton-cholesky', 'penalty': 'l2', 'C': 0.12}

In [34]:
y_linreg_rand_search_predict = random_search_lin_reg.predict(X_test)

print('Метрика f1 Логистической регрессии с применение GridSearchm {}'
      .format(round(f1_score(y_test, y_linreg_rand_search_predict), 2)))

Метрика f1 Логистической регрессии с применение GridSearchm 0.79


Оптимизация с помощью hyperopt

In [57]:
hp_dict = {
    'penalty': hp.choice('penalty', ['l2', 'none']),
    'solver': hp.choice('solver', ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']),
    'C': hp.loguniform('C', np.log(0.01), np.log(1)),
    'max_iter': hp.quniform('max_iter', 1000, 5000, 500) # в hyperopt попробуем еще дополнительно подобрать и этот параметр
}

def hp_optimization_func(params):
    
    param_dict = {
        'penalty': params['penalty'],
        'solver': params['solver'],
        'C': params['C'],
        'max_iter': int(params['max_iter'])
    }
    
    log_reg = LogisticRegression(
        **param_dict, random_state=42
    )
    
    score = cross_val_score(
        estimator=log_reg,
        X=X_train,
        y=y_train,
        cv=5,
        scoring='f1',
        n_jobs=-1
    )
    
    return -np.mean(score)


trial = Trials() # сохранимся, возможно придется дотюнить параметры

best = fmin(
    fn=hp_optimization_func,
    space=hp_dict,
    algo=tpe.suggest,
    trials=trial,
    max_evals=20,
    rstate=np.random.default_rng(42) 
)

100%|██████████| 20/20 [08:33<00:00, 25.69s/trial, best loss: -0.7801662680182477]


In [51]:
best

{'C': 0.07413032553845407, 'max_iter': 3000.0, 'penalty': 0, 'solver': 4}

In [53]:
# попробуем протестировать еще на 20 шагах

hp_dict2 = {
    'penalty': hp.choice('penalty', ['l2', 'none']),
    'solver': hp.choice('solver', ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']),
    'C': hp.loguniform('C', np.log(0.01), np.log(1)),
    'max_iter': hp.quniform('max_iter', 1000, 5000, 500) # в hyperopt попробуем еще дополнительно подобрать и этот параметр
}

best2 = fmin(
    fn=hp_optimization_func,
    space=hp_dict2,
    trials=trial,
    algo=tpe.suggest,
    max_evals=40,
    rstate=np.random.default_rng(42) 
)

100%|██████████| 40/40 [02:54<00:00,  8.72s/trial, best loss: -0.7815882524730513]


In [54]:
best2

{'C': 0.0916860657159772, 'max_iter': 3000.0, 'penalty': 0, 'solver': 3}

In [61]:
# обучим модель на полученных параметрах и посмотрим метрику на тестовых данных

log_reg_hp = LogisticRegression(
    penalty='l2',
    C=0.09,
    max_iter=3000,
    solver='sag',
    random_state=42
)

log_reg_hp.fit(X_train, y_train)

y_pred_log_reg_hp = log_reg_hp.predict(X_test)

print('f1 score для Логистической регрессии и алгоритма hyperopt {}'
      .format(np.round(f1_score(y_test, y_pred_log_reg_hp), 2))
)

f1 score для Логистической регрессии и алгоритма hyperopt 0.79


Оптимизируем параметры с помощью библиотеки optuna

In [64]:
# для начала определим функцию которая будет у нас в алгоритме
# создаем функцию для работы алгоритма optuna

def optuna_optimizer(trial):
    penalty = trial.suggest_categorical('penalty', ['l2', 'none'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga'])
    C = trial.suggest_float('C', 0.01, 1, step=0.01) # поиграем немного со значениями
    max_iter = trial.suggest_int('max_iter', 1000, 4000, step=250) # продолжим играть со значениями уже на итерациях :)
    
    log_reg = LogisticRegression(
        penalty=penalty,
        solver=solver,
        C=C,
        max_iter=max_iter,
        random_state=42  
        )
    
    score = cross_val_score(
        estimator=log_reg,
        X=X_train,
        y=y_train,
        cv=5,
        scoring='f1',
        n_jobs=-1
        )

    return np.mean(score)

study = optuna.create_study(study_name='LogReg_by_optuna', direction='maximize')

study.optimize(optuna_optimizer, n_trials=20)

[I 2024-08-22 21:59:58,056] A new study created in memory with name: LogReg_by_optuna
[I 2024-08-22 22:00:14,471] Trial 0 finished with value: 0.7652200898479752 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.6900000000000001, 'max_iter': 2250}. Best is trial 0 with value: 0.7652200898479752.
[I 2024-08-22 22:00:21,932] Trial 1 finished with value: 0.7165567903719422 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.51, 'max_iter': 3000}. Best is trial 0 with value: 0.7652200898479752.
[I 2024-08-22 22:00:24,221] Trial 2 finished with value: 0.7738689755891965 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.17, 'max_iter': 2500}. Best is trial 2 with value: 0.7738689755891965.
[I 2024-08-22 22:00:30,637] Trial 3 finished with value: 0.7165567903719422 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.5700000000000001, 'max_iter': 4000}. Best is trial 2 with value: 0.7738689755891965.
[I 2024-08-22 22:00:46,452] Trial 4 finished with value

In [65]:
# попробуем еще 20 итераций 

%time study.optimize(optuna_optimizer, n_trials=20)

[I 2024-08-22 22:05:28,407] Trial 20 finished with value: 0.7738689755891965 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.17, 'max_iter': 1250}. Best is trial 7 with value: 0.7784273109455763.
[I 2024-08-22 22:05:31,602] Trial 21 finished with value: 0.7777348723012718 and parameters: {'penalty': 'l2', 'solver': 'newton-cholesky', 'C': 0.13, 'max_iter': 2500}. Best is trial 7 with value: 0.7784273109455763.
[I 2024-08-22 22:05:35,214] Trial 22 finished with value: 0.7679035871915195 and parameters: {'penalty': 'l2', 'solver': 'newton-cholesky', 'C': 0.24000000000000002, 'max_iter': 2500}. Best is trial 7 with value: 0.7784273109455763.
[I 2024-08-22 22:05:38,282] Trial 23 finished with value: 0.7800725761717234 and parameters: {'penalty': 'l2', 'solver': 'newton-cholesky', 'C': 0.08, 'max_iter': 2250}. Best is trial 23 with value: 0.7800725761717234.
[I 2024-08-22 22:05:41,266] Trial 24 finished with value: 0.7777461288275932 and parameters: {'penalty': 'l2', 'solver': '

CPU times: total: 266 ms
Wall time: 1min 59s


In [66]:
study.best_params

{'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.09, 'max_iter': 3750}

In [68]:
# визуализируем результаты оптимизации
optuna.visualization.plot_optimization_history(study)

In [71]:

log_reg_optuna = LogisticRegression(**study.best_params, random_state=42)
log_reg_optuna.fit(X_train, y_train)
y_pred_optuna_logreg = log_reg_optuna.predict(X_test)

optuna_f1_score_logreg = np.round(f1_score(y_test, y_pred_optuna_logreg), 2)

print('Результаты метрики f1 на оптимизаторе optuna для алгоритма LogisticRegression {}'
      .format(optuna_f1_score_logreg))

Результаты метрики f1 на оптимизаторе optuna для алгоритма LogisticRegression 0.79


## Оптимизация модели RandomForestClassifier

In [7]:
# создадим базовую модель и обучим ее

rnd_clf = RandomForestClassifier(random_state=42)

rnd_clf.fit(X_train, y_train)

y_pred_rnd = rnd_clf.predict(X_test)

rnd_f1_score = np.round(f1_score(y_test, y_pred_rnd), 2)

print('Метрика f1 на базовой модели RandomForestClassifier {}'
      .format(rnd_f1_score))

Метрика f1 на базовой модели RandomForestClassifier 0.82


оптимизация с помощью GridSearchCV

In [53]:
# создадим переменную estimator в которой сохраним базовые настройки модели

estimator = RandomForestClassifier(random_state=42)

In [58]:
param_rand_forest_dict = {
                    'min_samples_leaf': list(np.linspace(5, 100, 10, dtype=int)),
                    'criterion': ['gini', 'entropy'],
                    'max_depth': list(np.linspace(1, 30, 10, dtype=int))
}

grid_rnd_frst = GridSearchCV(
        estimator=estimator,
        param_grid=param_rand_forest_dict,
        cv=5,
        n_jobs=-1,
)

%time grid_rnd_frst.fit(X_train, y_train)

CPU times: total: 5.11 s
Wall time: 1min 27s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 4, 7, 10, 13, 17, 20, 23, 26, 30],
                         'min_samples_leaf': [5, 15, 26, 36, 47, 57, 68, 78, 89,
                                              100]})

In [59]:
grid_rnd_frst.best_params_

{'criterion': 'entropy', 'max_depth': 17, 'min_samples_leaf': 5}

In [60]:
# обучим на подобранных параметрах и проверим метрику

y_grid_search_rnd = grid_rnd_frst.predict(X_test)

rnd_grid_f1_score = round(f1_score(y_test, y_grid_search_rnd), 2)

print('Метрика f1 на параметрах подобранных GridSearch для RandomForestClassifier {}'
      .format(rnd_grid_f1_score))

Метрика f1 на параметрах подобранных GridSearch для RandomForestClassifier 0.81


оптимизация с помощью RandomizedSearchCV

In [29]:

param_distr_dict = {'min_samples_leaf': list(np.linspace(5, 100, 20, dtype=int)), 
                    'criterion': ['gini', 'entropy'],
                    'max_depth': list(np.linspace(1, 30, 30, dtype=int))
}

rand_cv_rnd_frst = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_distr_dict,
        scoring='f1',
        cv=5,
        n_jobs=-1,
        random_state=42)

%time rand_cv_rnd_frst.fit(X_train, y_train)

CPU times: total: 1.3 s
Wall time: 7.37 s


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30],
                                        'min_samples_leaf': [5, 10, 15, 20, 25,
                                                             30, 35, 40, 45, 50,
                                                             55, 60, 65, 70, 75,
                                                             80, 85, 90, 95,
                                                             100]},
                   random_state=42, scoring='f1')

In [30]:
rand_cv_rnd_frst.best_params_

{'min_samples_leaf': 5, 'max_depth': 14, 'criterion': 'entropy'}

In [33]:

y_pred_rand_cv = rand_cv_rnd_frst.predict(X_test)

rand_cv_f1 = round(f1_score(y_test, y_pred_rand_cv), 2)

print('Метрика f1 на параметрах подобранных GridSearch для RandomForestClassifier {}'
      .format(rand_cv_f1))

Метрика f1 на параметрах подобранных GridSearch для RandomForestClassifier 0.82


Оптимизация RandomForestClassifier с помощью библиотеки hyperopt

In [47]:
hp_dict = {'n_estimators': hp.quniform('n_estimators', 100, 1000, 100),
           'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 50, 1),
           'criterion': hp.choice('criterion', ['gini', 'entropy']),
           'max_depth': hp.quniform('max_depth', 1, 30, 1)
}

def hp_optim_rand_forest(params):
    
    param_dict = {
        'min_samples_leaf': int(params['min_samples_leaf']),
        'criterion': params['criterion'],
        'max_depth': int(params['max_depth'])
    }
    
    rand_forest = RandomForestClassifier(**param_dict, random_state=42)
    
    score = cross_val_score(
        estimator=rand_forest,
        X=X_train,
        y=y_train,
        cv=5,
        scoring='f1',
        n_jobs=-1
    )
    
    return -np.mean(score)


trial = Trials() # сохранимся, возможно придется дотюнить параметры

best = fmin(
    fn=hp_optim_rand_forest,
    space=hp_dict,
    algo=tpe.suggest,
    trials=trial,
    max_evals=20,
    rstate=np.random.default_rng(42) 
)

100%|██████████| 20/20 [00:19<00:00,  1.00trial/s, best loss: -0.792642026740545]


In [50]:
best = fmin(
    fn=hp_optim_rand_forest,
    space=hp_dict,
    algo=tpe.suggest,
    trials=trial,
    max_evals=40,
    rstate=np.random.default_rng(42) 
)

100%|██████████| 40/40 [00:25<00:00,  1.28s/trial, best loss: -0.8016030765645095]


In [51]:
# выведем лучшие параметры модели
best

{'criterion': 1,
 'max_depth': 17.0,
 'min_samples_leaf': 5.0,
 'n_estimators': 200.0}

In [63]:
hp_rand_forest = RandomForestClassifier(
                       n_estimators=200,
                       criterion='entropy',
                       max_depth=17,
                       min_samples_leaf=5,
                       random_state=42)

hp_rand_forest.fit(X_train, y_train)

y_pred_hp = hp_rand_forest.predict(X_test)

hp_f1_score = round(f1_score(y_test, y_pred_hp), 2)

print('Метрика f1 на параметрах подобранных алгоритмом hyperopt для RandomForestClassifier {}'
      .format(hp_f1_score))

Метрика f1 на параметрах подобранных алгоритмом hyperopt для RandomForestClassifier 0.82


Оптимизиция RandomForestClassifier на алгоритме optuna

In [69]:
# создаем функцию для алгоритма и оптимизируем параметры
def optimize_random_forest(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=50)
    criterion = trial.suggest_categorical('criterion', ['entropy', 'gini'])
    max_depth = trial.suggest_int('max_depth', 1, 20, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 35, step=1)
    
    estimator = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    score = cross_val_score(
        estimator=estimator,
        X=X_train,
        y=y_train,
        cv=5,
        n_jobs=-1,
        scoring='f1'
    )
    
    return np.mean(score)

study = optuna.create_study(study_name='RandomForestClassifier_Optimization', direction='maximize')

best = study.optimize(optimize_random_forest, n_trials=20)

[I 2024-08-24 10:45:32,426] A new study created in memory with name: RandomForestClassifier_Optimization
[I 2024-08-24 10:45:36,648] Trial 0 finished with value: 0.7495370649671032 and parameters: {'n_estimators': 700, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.7495370649671032.
[I 2024-08-24 10:45:40,156] Trial 1 finished with value: 0.767571782318411 and parameters: {'n_estimators': 450, 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 19}. Best is trial 1 with value: 0.767571782318411.
[I 2024-08-24 10:45:43,322] Trial 2 finished with value: 0.7769818635068376 and parameters: {'n_estimators': 350, 'criterion': 'entropy', 'max_depth': 19, 'min_samples_leaf': 20}. Best is trial 2 with value: 0.7769818635068376.
[I 2024-08-24 10:45:44,826] Trial 3 finished with value: 0.7232796453833517 and parameters: {'n_estimators': 650, 'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 19}. Best is trial 2 with value: 0.776981863506

In [70]:
# повторим еще 20 шагов

best = study.optimize(optimize_random_forest, n_trials=40)

[I 2024-08-24 10:48:13,145] Trial 20 finished with value: 0.785629425045913 and parameters: {'n_estimators': 400, 'criterion': 'entropy', 'max_depth': 17, 'min_samples_leaf': 16}. Best is trial 12 with value: 0.8036811798198636.
[I 2024-08-24 10:48:20,456] Trial 21 finished with value: 0.8023451381591944 and parameters: {'n_estimators': 550, 'criterion': 'entropy', 'max_depth': 18, 'min_samples_leaf': 5}. Best is trial 12 with value: 0.8036811798198636.
[I 2024-08-24 10:48:27,138] Trial 22 finished with value: 0.7940625965636924 and parameters: {'n_estimators': 600, 'criterion': 'entropy', 'max_depth': 18, 'min_samples_leaf': 9}. Best is trial 12 with value: 0.8036811798198636.
[I 2024-08-24 10:48:33,795] Trial 23 finished with value: 0.8014697027649902 and parameters: {'n_estimators': 550, 'criterion': 'entropy', 'max_depth': 14, 'min_samples_leaf': 5}. Best is trial 12 with value: 0.8036811798198636.
[I 2024-08-24 10:48:36,877] Trial 24 finished with value: 0.7896995411355273 and par

In [71]:
optuna.visualization.plot_optimization_history(study)

In [75]:
rnd_frst_optuna = RandomForestClassifier(
    **study.best_params,
    random_state=42
)

rnd_frst_optuna.fit(X_train, y_train)

y_pred_best_optuna = rnd_frst_optuna.predict(X_test)
optuna_f1_random_frst = round(f1_score(y_test, y_pred_best_optuna), 2)

print('Метрика f1 на параметрах подобранных алгоритмом optuna для RandomForestClassifier {}'
      .format(optuna_f1_random_frst))

Метрика f1 на параметрах подобранных алгоритмом optuna для RandomForestClassifier 0.83


##### Общий вывод:
- Провели оптимизацию 2х моделей LogisticRegression и RandomForestClassifier
- Использовали 4 алгоритма оптимизации GridSearch и RamdomSearch, а также продвинутые алгориты Hyperopt и Optuna
- Использовали кросс-валидацию во всех случаях
- Для проверки использовали метрику f1_score
- По итогу можно сказать, что начальные настройки алгоритмов в целом показывают достойные результаты уже, что называется, из коробки
- Метрику удалось улучшить лишь на 0.01 при применении RandomForestClassifier и алгоритма optuna
- RandomForestClassifier в целом на тестовой выборки показывает лучнее показание f1

P.S. Лично мне понравился больше всего алгоритм optuna, очень понятный синтаксис, есть визуализация результатов, НАМНОГО быстрее чем какой нибудь GridSearch, есть можность задать гораздо больше параметров для поиска и при этом не умереть от старости.